# Day 14: Week 2の復習とミニプロジェクト

## Learning Objectives
- Week 2の内容を復習する
- 画像処理パイプラインを構築する

## Week 2 復習

**学んだ内容**:
- Day 8: 2次元配列と画像表現
- Day 9: 畳み込みとフィルタ処理（前半）
- Day 10: 畳み込みとフィルタ処理（後半）
- Day 11: モルフォロジー処理
- Day 12: ヒストグラム処理
- Day 13: 幾何学変換

## 画像処理パイプライン

In [ ]:
class ImagePipeline:
    """画像処理パイプライン
    
    複数のフィルタを順番に適用
    """
    
    def __init__(self):
        self.filters = []
    
    def add_filter(self, filter_func, name=None):
        """フィルタを追加"""
        self.filters.append((filter_func, name))
        return self
    
    def process(self, image):
        """全フィルタを順番に適用"""
        result = image
        history = [("Original", image)]
        
        for filter_func, name in self.filters:
            result = filter_func(result)
            if name:
                history.append((name, result))
        
        return result, history

# 畳み込み処理のヘルパー関数
def apply_convolution(image, kernel):
    """畳み込み処理を実行"""
    
    h, w = len(image), len(image[0])
    k_h, k_w = len(kernel), len(kernel[0])
    
    result = [[0 for _ in range(w)] for _ in range(h)]
    
    # パディングなしの畳み込み（境界部は処理しない）
    for y in range(1, h-1):
        for x in range(1, w-1):
            value = 0
            for ky in range(k_h):
                for kx in range(k_w):
                    value += image[y + ky - 1][x + kx - 1] * kernel[ky][kx]
            result[y][x] = int(value)
    
    return result

# ノイズ低減パイプライン
pipeline = ImagePipeline()
pipeline.add_filter(lambda img: [[min(255, int(p * 1.2)) for p in row] for row in img], "Brightness x1.2")
pipeline.add_filter(lambda img: apply_convolution(img, [[1/9]*3 for _ in range(3)]), "Gaussian Blur")

# テスト
test_img = [
    [50, 60, 70, 80, 90],
    [60, 70, 80, 90, 100],
    [70, 80, 90, 100, 110],
    [80, 90, 100, 110, 120]
]

print("元の画像:")
for row in test_img:
    print(row)

result, history = pipeline.process(test_img)

for name, img in history:
    print(f"\n{name}:")
    for row in img:
        print([int(v) for v in row])

## エッジ検出パイプライン

In [ ]:
def edge_detection_pipeline(image):
    """エッジ検出パイプライン
    
    1. グレースケール変換
    2. ガウシアンフィルタ（平滑化）
    3. Sobelフィルタ（エッジ検出）
    4. 2値化
    """
    # 1. すでにグレースケールと仮定
    img = image
    
    # 2. ガウシアンフィルタ
    gaussian = [[1/16, 2/16, 1/16], [2/16, 4/16, 2/16], [1/16, 2/16, 1/16]]
    img = apply_convolution(img, gaussian)
    
    # 3. Sobelフィルタ
    sobel_x = [[-1, 0, 1], [-2, 0, 2], [-1, 0, 1]]
    sobel_y = [[-1, -2, -1], [0, 0, 0], [1, 2, 1]]
    
    gx = apply_convolution(img, sobel_x)
    gy = apply_convolution(img, sobel_y)
    
    # 勾配の大きさ
    h, w = len(img), len(img[0])
    magnitude = [[0 for _ in range(w)] for _ in range(h)]
    for y in range(h):
        for x in range(w):
            magnitude[y][x] = int((gx[y][x]**2 + gy[y][x]**2)**0.5)
    
    # 4. 2値化
    threshold_val = 50
    binary = [[255 if magnitude[y][x] >= threshold_val else 0 for x in range(w)] for y in range(h)]
    
    return binary

# テスト（エッジのある画像）
edge_img = [
    [255, 255, 255, 255, 255],
    [255, 255, 255, 255, 255],
    [255, 255, 255, 255, 255],
    [0, 0, 0, 0, 0],
    [0, 0, 0, 0, 0]
]

print("元の画像（上:白、下:黒）:")
for row in edge_img:
    print(row)

edges = edge_detection_pipeline(edge_img)
print("\nエッジ検出結果:")
for row in edges:
    print(row)

print("\n境界線（255）が検出されている")

## Week 2 振り返り

### 習得したスキル:
- [ ] 2次元配列で画像を表現できる
- [ ] 畳み込み演算を理解した
- [ ] 様々なフィルタを実装した
- [ ] モルフォロジー処理を理解した
- [ ] ヒストグラム解析ができる
- [ ] 幾何学変換を実装した
- [ ] 画像処理パイプラインを構築した

**お疲れ様でした！** Week 2完了！